# Algo TP3 

# Ex 1. Chaine de 0 et 1 qui resoud un UBQP
---

### Reading Bit file ex `graphe_.txt`

In [1]:
# Read file
def read_file_ubgp(file_path):
    f = open(file_path, "r")
    
    content = f.read()
    content = content.replace("\n", "")
    content_list = content.split(" ")
    content_list = list(map(int, content_list))
    
    n = content_list[0]
    p = content_list[1]
    Q = np.array(content_list[2:])
    Q.shape = ((n,n))
    
    f.close()
    return(n,p,Q)

In [2]:
import numpy as np

def create_rand_bit_vector(n):
    return [np.random.choice([0,1]) for i in range(n)]

### Neighbours

In [3]:
# Unconstrained Binary Quadratic Problem
def UBGP(matrix,X):        
    #result = np.apply_along_axis((lambda row: row*X), axis=1, arr=matrix).sum(axis=1)
    res = 0
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            res += matrix[i,j]*X[i]*X[j]
        
    return res

In [4]:
def voisins(s):
    voisins = []
    temp = []
    for i in range(len(s)):
        temp = s.copy()
        temp[i] = int(not temp[i])
        voisins.append(temp) 
    return voisins

In [5]:
def meilleur_voisin(X,p,Q):
    solutions  = []
    neighbours = []
    neighbours = voisins(X)
    solutions.append(neighbours[0])
    for i in range(1,len(neighbours)):
        if(sum(neighbours[i])>=p and UBGP(Q,neighbours[i])==UBGP(Q,solutions[0])):
            solutions.append(neighbours[i])
        elif(sum(neighbours[i])>=p and UBGP(Q,neighbours[i])<UBGP(Q,solutions[0])):
            solutions = []
            solutions.append(neighbours[i])
        #print(solutions)
    randomRow = np.random.randint(len(solutions), size=1)
    return (solutions[randomRow[0]])

In [6]:
def meilleur(s1, s2, Q): #MIN OPT?
    return(UBGP(Q,s1)<UBGP(Q,s2))

In [7]:
def voisins_non_Tabou(s,tabou):
    voisins = []
    temp = []
    for i in range(len(s)):
        temp = s.copy()
        temp[i] = int(not temp[i])
        if(temp not in tabou):
            voisins.append(temp) 
    return voisins

In [8]:
def meilleur_voisin_non_tabou(X,Q,p,t):
    solutions  = []
    neighbours = []
    neighbours = voisins_non_Tabou(X,t)
    s = neighbours[0]
    for i in range(1,len(neighbours)):
        if(UBGP(Q,neighbours[i])<=UBGP(Q,s)):
            s = neighbours[i]
    return s

### Algorithms 

In [9]:
def steepest_hill_climbing(s,Q,p,MAX_depl):
    nb_depl = 0
    stop = False
    print("Solution inital : ", s)
    while(True):
        new_s = meilleur_voisin(s,p,Q)
        print("meilleur voisin ",nb_depl," : ",new_s," val ",UBGP(Q,new_s))
        if(meilleur(new_s,s,Q)):
            s = new_s
        else:
            stop = True # optimum local
        nb_depl = nb_depl + 1
        if(stop or nb_depl == MAX_depl):
            break
    print("Solution atteinte ",s,", avec nb deplacement: ", nb_depl)
    return s

In [95]:
from queue import Queue

def tabou(s,Q,p,MAX_depl,k):
    #tabou = Queue(maxsize = k)
    tabou = []
    nb_depl = 0
    msol = s
    stop = False
    print("Solution inital : ", s)
    while(True):
        if(voisins_non_Tabou(s,tabou)): # not empty
            new_s = meilleur_voisin_non_tabou(s,Q,p,tabou) #plus de voisin non tabou
            print("meilleur voisin ", nb_depl," : ",new_s," val ",UBGP(Q,new_s))
        else:
            stop = True
        #tabou.put(s)
        s = new_s
        tabou.append(s)
        tabou = tabou[:k] #max size 
        if(meilleur(new_s, msol, Q)):
            msol = new_s #stockage meilleure solution courante
        nb_depl = nb_depl + 1
        if(stop or nb_depl == MAX_depl):
            break
    print("Solution atteinte ",msol,", avec nb deplacement: ", nb_depl,"\n Tabou : [",tabou,"]")
    return msol        

### Main Bit

In [97]:
#---- read from file vector size N, verification P and matrix Q 
n,p,Q = read_file_ubgp("data/graphe12345bis.txt")
# print(n,p,Q)

MAX_essais = 10
MAX_depl = 15

solutions = []
solutions_ubgp = []
s = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
s = create_rand_bit_vector(n)


#---- for each essais make max depl to find MIN opt in Steepest Hill
print("-----------STEEPEST HILL CLIMB---------")
for i in range(MAX_essais):
    print("Essais ", i)
    solutions.append(steepest_hill_climbing(s,Q,p,MAX_depl))
    s = create_rand_bit_vector(n)
    
#---- print all essais solutions
for sol in solutions:
    ubgp = UBGP(Q,sol)
    solutions_ubgp.append(ubgp)
    print(sol,": ",ubgp)

#---- from all essais solutions which one is best
optimal = solutions[solutions_ubgp.index(min(solutions_ubgp))]
print("Optimal : ",optimal, UBGP(Q,optimal))

#---- Tabou
print("-----------TABOU----------")
k = 20
sol_tabou = tabou(s,Q,p,MAX_depl,k)
print("Optimal Tabou solution: ",sol_tabou,UBGP(Q,sol_tabou))



-----------STEEPEST HILL CLIMB---------
Essais  0
Solution inital :  [0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0]
meilleur voisin  0  :  [0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0]  val  4
meilleur voisin  1  :  [0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0]  val  -8
meilleur voisin  2  :  [0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0]  val  -12
meilleur voisin  3  :  [0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0]  val  -16
meilleur voisin  4  :  [1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0]  val  -16
Solution atteinte  [0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0] , avec nb deplacement:  5
Essais  1
Solution inital :  [0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0]
meilleur voisin  0  :  [0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0]  val  40
meilleur voisin  1  :  [0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0]  val  20
meilleur voisin  2  :  [0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0]  val  0
meilleur voisin  3  :  [0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0]  val  -12
meilleur voisin

# Ex 2. Traveling Salesman
---

### Reading TSP file ex. `tsp_.txt`

In [83]:
# Read file
def read_file_tsp(file_path):
    f = open (file_path , 'r')
    l = [line.split() for line in f]
    #l.astype(int)
    #l = list(map(int, l))
    n = l[0][0]
    villes = l[1:]
    f.close()
    return(n,villes)

In [84]:
import numpy as np

# returns a random path from villes[0] city
def chemin_random(villes):
    chemin = []
    chemin.append(villes[0]) # first city static 
    copy_villes = villes.copy()
    for i in range(1,len(copy_villes)):
        randomRow = np.random.randint(low=1,high=len(copy_villes), size=1)
        chemin.append(copy_villes[randomRow[0]])
        del copy_villes[randomRow[0]]
    chemin.append(villes[0]) # return at the same city
    return chemin

###  Calculating distance 

In [85]:
import math

def eucledian(ville1, ville2):
    return math.sqrt((int(ville1[1]) - int(ville2[1]))**2 + (int(ville1[2]) - int(ville2[2]))**2)

In [86]:
def distance(chemin):
    distance = 0
    for i in range(0,len(chemin)-1):
        distance = distance + eucledian(chemin[i],chemin[i+1])
    return distance

In [87]:
def voisin_tsp(chemin): 
    n = len(chemin)
    voisins = []
    for i in range(1,n-1):
        for j in range(i+1,n-1):
            chemin_copy =  chemin.copy()
            # replace city i with city j 
            temp = chemin_copy[i].copy()
            chemin_copy[i] = chemin_copy[j]
            chemin_copy[j] = temp
            
            voisins.append(chemin_copy)
    return voisins

In [88]:
def meilleur_voisin_tsp(chemin):
    voisins = voisin_tsp(chemin)
    msol = voisins[0]
    for i in range(1,len(voisins)):
        if(distance(voisins[i]) < distance(msol)):
            msol = voisins[i]
    return (msol)

In [89]:
def meilleur_tsp(chemin1, chemin2):
    if(len(chemin1) != len(chemin2)):
        print("ERROR : NOT SAME LENGHT OF CHEMINS")
    else:
        d1 = 0
        d2 = 0
        for i in range(0,len(chemin1)-1):
            d1 = d1 + eucledian(chemin1[i], chemin1[i+1])
            d2 = d2 + eucledian(chemin2[i], chemin2[i+1])
    return d1 < d2

In [102]:
def voisins_non_tabou_tsp(chemin, tabou): 
    n = len(chemin)
    voisins = []
    for i in range(1,n-1):
        for j in range(i+1,n-1):
            chemin_copy =  chemin.copy()
            
            # replace city i with city j 
            temp = chemin_copy[i].copy()
            chemin_copy[i] = chemin_copy[j]
            chemin_copy[j] = temp
            
            if(chemin_copy not in tabou):
                voisins.append(chemin_copy)
    return voisins

In [103]:
def meilleur_voisin_non_tabou_tsp(chemin, tabou):
    voisins = voisins_non_tabou_tsp(chemin, tabou)
    msol = voisins[0]
    for i in range(1,len(voisins)):
        if(distance(voisins[i]) < distance(msol)):
            msol = voisins[i]
    return (msol)

### Steepest Hill Climbing for TSP

In [104]:
def steepest_hill_climbing_tsp(s,MAX_depl):
    nb_depl = 0
    stop = False
    print("Solution initial ",s)
    while(True):
        new_s = meilleur_voisin_tsp(s)
        print("meilleur voisin ",nb_depl," : \n",new_s,"\n val ",distance(new_s))
        if(meilleur_tsp(new_s,s)):
            s = new_s
        else:
            stop = True # optimum local
        nb_depl = nb_depl+1
        if(stop or nb_depl == MAX_depl):
            break;
    print("Solution atteinte ",s, "\n avec nb deplacement: ", nb_depl,"val ", distance(s))
    return s

### Tabou TSP

In [114]:
def tabou_tsp(s, MAX_depl, k):
    tabou = []
    nb_depl = 0
    msol = s
    stop = False
    print("Solution initial ",s)
    while(True):
        if(voisins_non_tabou_tsp(s,tabou)):
            new_s = meilleur_voisin_non_tabou_tsp(s,tabou) #plus de voisin non tabou
            print("meilleur voisin ", nb_depl," : ",new_s," val ",distance(new_s))
        else:
            stop = True
        s = new_s
        tabou.append(s)
        tabou = tabou[:k] #max size 
        if(meilleur_tsp(new_s, msol)):
            msol = new_s #stockage meilleure solution courante
        nb_depl = nb_depl + 1
        if(nb_depl == MAX_depl or stop):
            break
    print("Solution atteinte ",msol, "\n avec nb deplacement: ", nb_depl," val ", distance(msol),"\n Tabou : ",tabou)
    return msol

### Main TSP

In [118]:
#---- read from file
n, villes = read_file_tsp("data/tsp5.txt")
#print(n,villes)


#---- add first static city/storage
villes_with_start = [['0','0','0']]
for i in range(len(villes)):
    villes_with_start.append(villes[i])
print(villes_with_start)


#---- random initial solution
chemin = chemin_random(villes_with_start)
d = distance(chemin)
print(chemin, d)


# #---- for each essais make max depl to find MIN opt in Steepest Hill
# print("-----------STEEPEST HILL CLIMB---------")
# MAX_essais = 3
# solutions = []
# for i in range(MAX_essais):
#     print("Essais ", i)
#     solutions.append(steepest_hill_climbing_tsp(chemin,MAX_depl))
#     chemin = chemin_random(villes_with_start)
    
# #---- print all essais solutions
# solutions_distance = []
# for sol in solutions:
#     distance_sol = distance(sol)
#     solutions_distance.append(distance_sol)
#     print(sol,": ",distance_sol)

# #---- from all essais solutions which one is best
# optimal = solutions[solutions_distance.index(min(solutions_distance))]
# print("Optimal in all essais : ",optimal, distance(optimal))

#---- Tabou
print("-----------TABOU----------")
k = 3
MAX_depl = 5
print(tabou_tsp(chemin,MAX_depl,k))

[['0', '0', '0'], ['1', '40', '50'], ['2', '25', '85'], ['3', '22', '75'], ['4', '22', '85'], ['5', '20', '80']]
[['0', '0', '0'], ['1', '40', '50'], ['3', '22', '75'], ['2', '25', '85'], ['4', '22', '85'], ['5', '20', '80'], ['0', '0', '0']] 196.12466980422548
-----------TABOU----------
Solution initial  [['0', '0', '0'], ['1', '40', '50'], ['3', '22', '75'], ['2', '25', '85'], ['4', '22', '85'], ['5', '20', '80'], ['0', '0', '0']]
meilleur voisin  0  :  [['0', '0', '0'], ['1', '40', '50'], ['3', '22', '75'], ['5', '20', '80'], ['4', '22', '85'], ['2', '25', '85'], ['0', '0', '0']]  val  197.20764132344294
meilleur voisin  1  :  [['0', '0', '0'], ['1', '40', '50'], ['3', '22', '75'], ['2', '25', '85'], ['4', '22', '85'], ['5', '20', '80'], ['0', '0', '0']]  val  196.12466980422548
meilleur voisin  2  :  [['0', '0', '0'], ['1', '40', '50'], ['3', '22', '75'], ['4', '22', '85'], ['2', '25', '85'], ['5', '20', '80'], ['0', '0', '0']]  val  197.3702663000459
meilleur voisin  3  :  [['0', 